In [4]:
import pandas as pd
import numpy as np
import boto3
import sagemaker
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import tree
import warnings
warnings.filterwarnings('ignore')
import time
from sklearn.preprocessing import LabelEncoder

data = "s3://pollution-predictor-1/uspollution_pollution_us_2000_2016.csv"
df = pd.read_csv(data)
column_names=["State Code","County Code","Site Num","Address","State","County","City","Date Local",
             "NO2 Mean","NO2 1st Max Value","NO2 1st Max Hour","NO2 AQI","O3 Mean","O3 1st Max Value",
              "O3 1st Max Hour","O3 AQI","SO2 Mean","SO2 1st Max Value","SO2 1st Max Hour","SO2 AQI",
              "CO Mean","CO 1st Max Value","CO 1st Max Hour","CO AQI"]

del df['Unnamed: 0']
del df['NO2 Units']
del df['O3 Units']
del df['SO2 Units']
del df['CO Units']

states=[]
for i in df["State"]:
    if i not in states:
        states.append(i)
print(states,len(states))

['Arizona', 'California', 'Colorado', 'District Of Columbia', 'Florida', 'Illinois', 'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Michigan', 'Missouri', 'New Jersey', 'New York', 'North Carolina', 'Oklahoma', 'Pennsylvania', 'Texas', 'Virginia', 'Massachusetts', 'Nevada', 'New Hampshire', 'Tennessee', 'South Carolina', 'Connecticut', 'Iowa', 'Maine', 'Maryland', 'Wisconsin', 'Country Of Mexico', 'Arkansas', 'Oregon', 'Wyoming', 'North Dakota', 'Idaho', 'Ohio', 'Georgia', 'Delaware', 'Hawaii', 'Minnesota', 'New Mexico', 'Rhode Island', 'South Dakota', 'Utah', 'Alabama', 'Washington', 'Alaska'] 47


In [ ]:
le = LabelEncoder()

df['State'] = le.fit_transform(df['State'])
state_map = dict(zip(le.classes_, le.transform(le.classes_)))

df['County'] = le.fit_transform(df['County'])
county_map = dict(zip(le.classes_, le.transform(le.classes_)))

df['City'] = le.fit_transform(df['City'])
city_map = dict(zip(le.classes_, le.transform(le.classes_)))

df['Address'] = le.fit_transform(df['Address'])
address_map = dict(zip(le.classes_, le.transform(le.classes_)))

df['Date Local'] = le.fit_transform(df['Date Local'])
datelocal_map = dict(zip(le.classes_, le.transform(le.classes_)))

def replace_with_nan(df_col, labels):
    for key in labels.keys():
        if type(key) == float:
            df_col = df_col.replace(labels[key], np.NaN)
    return df_col

df['State'] = replace_with_nan(df['State'], state_map)
df['County'] = replace_with_nan(df['County'], county_map)
df['City'] = replace_with_nan(df['City'], city_map)
df['Address'] = replace_with_nan(df['Address'], address_map)
df['Date Local'] = replace_with_nan(df['Date Local'], datelocal_map)


In [ ]:
from sklearn.impute import SimpleImputer
df_copy = df.copy(deep = True)
imputer = SimpleImputer(strategy='mean')

imputed = imputer.fit_transform(df)

df_imputed = pd.DataFrame(imputed, columns = column_names)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler()

scaled_array = scaler.fit_transform(df_imputed.values)

df_scaled = pd.DataFrame(scaled_array, columns = column_names)